<h1 style="font-family:Times New Roman;">Поиск населенных пунктов по части названия в OpenStreerMap</h1>

*Nikolay Sitala 2023, nsitala@ukr.net* 
>Данные по материалам сайта https://ua.transportica.com

In [ ]:
# Imports for JupyterLite
try:
    import piplite
    await piplite.install(['ipywidgets'])
    await piplite.install(['panel'])
except ImportError:
    pass

import ipywidgets as widgets
text_ = widgets.Text(
    value='Киев',
    placeholder='Type something',
    description='Поиск подстроки:',
    disabled=False,
    layout = widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)
display(text_)

<h2 style="font-family: 'Times New Roman'">Скрипт поиска</h2>

In [37]:
import json
import pandas as pd
import panel as pn

from js import WebSocket, console
from pyodide.ffi.wrappers import add_event_listener

pn.extension('perspective')

i = 0

def send(*args):
    add_event_listener(socket, 'message', message)

    payload = {"lang": "ua", "text": text_.value, "country": "UA", "event": "completeCity", "requestNumber": 1}
    data = json.dumps(payload)
    
    socket.send(data)

def message(event):
    global i
    if type(event.data) == str and i == 0:
        i += 1
        js = json.loads(event.data)
        p = [[], [], [], [], [], [], []]

        for b in js['data']['result']:
            p[0].append(b['osmId'])
            p[1].append(b['name'])
            p[2].append(b['state'])
            p[3].append(b['lat'])
            p[4].append(b['lon'])
            p[5].append(b['city'])

            admin_level = b['hierarchy']
            if len(admin_level) > 1: 
                p[6].append(admin_level[1]['default'])
            else:
                p[6].append(None)

        data_ = {'osmId': p[0], 'city': p[5], 'level': p[6], 'name': p[1], 'state': p[2], 'lat': p[3], 'lon': p[4]}
        fr = pd.DataFrame.from_dict(data_)
        print("ok")
        display(pn.pane.Perspective(fr, width=1000, height=600))

socket = WebSocket.new("wss://complete.transportica.com:443")
add_event_listener(socket, 'open', send)
add_event_listener(socket, 'message', message)

ok


Perspective(DataFrame, columns=['index', 'osmId', ...], height=600, sizing_mode='fixed', width=1000)